In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import pandas as pd
import os
import cv2  
from tqdm import tqdm
from utils import show_image, main_color

from sklearn.cluster import KMeans
from collections import Counter
from skimage.color import rgb2lab, deltaE_cie76

In [2]:
from datetime import datetime
from packaging import version

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import rcParams

from scipy import ndimage 

print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.6.0


In [3]:
train_path = "data/train"
test_path = "data/test"

train = os.listdir(train_path)
test = os.listdir(test_path)

In [4]:
train_image = [cv2.cvtColor(cv2.imread(os.path.join(train_path, image)), cv2.COLOR_BGR2RGB)  for image in train ]

In [5]:
test_image = [cv2.cvtColor(cv2.imread(os.path.join(test_path, image)), cv2.COLOR_BGR2RGB)  for image in test ]

In [7]:
trian_label = pd.read_csv("data/imagenames.csv")

#color_total_train,weight_total_train = main_color(train_image[0:5],3,False)


100%|██████████| 5/5 [00:05<00:00,  1.16s/it]


In [6]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

# load the model
model = ResNet50(weights='imagenet')

In [50]:
test_object = []
test_weight = []
for i in tqdm(range(len(test))):
    imgpath = test_path+"/"+test[i]
    img = image.load_img(imgpath, target_size=(224, 224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    k_img = preprocess_input(img)
    preds = model.predict(k_img)

    
    detect_object = [pred[1] for pred in decode_predictions(preds, top=5)[0]]
    detect_weight = [pred[2] for pred in decode_predictions(preds, top=5)[0]]

    test_object.append(detect_object)
    test_weight.append(detect_weight)




100%|██████████| 1200/1200 [01:00<00:00, 19.93it/s]


In [51]:
with_object_test = pd.DataFrame({"object":test_object,"object_weight":test_weight})

In [ ]:
with_object_test

In [52]:
test_color = pd.read_csv("data/with_color_test.csv")

In [53]:
test_color["object"] = with_object_test["object"]
test_color["object_weight"] = with_object_test["object_weight"]

In [59]:
train_color.to_csv("data/train_features.csv",index = False)

In [61]:
train_color

,id,x,y,color,color_weight,object,object_weight,cluster
0,IMG2744_1,-9.380678,3.58272,[[105.42161563 107.14448812 100.08701536]\n [ ...,"[0.6348333333333334, 0.28917916666666665, 0.07...","[patio, lakeside, bannister, mobile_home, brass]","[0.5601092, 0.14016706, 0.022413984, 0.0214102...",IMG2744
1,IMG2744_2,-9.380678,3.58272,[[ 70.89338335 69.35952808 46.43450226]\n [2...,"[0.24418333333333334, 0.383675, 0.372141666666...","[lakeside, fountain, snowplow, cab, patio]","[0.19650029, 0.09605682, 0.07403735, 0.0641981...",IMG2744
2,IMG2744_3,-9.380678,3.58272,[[102.4665223 104.16690078 106.75012681]\n [ ...,"[0.38762083333333336, 0.21436666666666668, 0.3...","[planetarium, recreational_vehicle, patio, dom...","[0.19081369, 0.18522698, 0.15903644, 0.1425865...",IMG2744
3,IMG2744_4,-9.380678,3.58272,[[172.34591855 173.62166457 174.12597505]\n [ ...,"[0.14006666666666667, 0.80195, 0.0579833333333...","[patio, prison, vault, sliding_door, library]","[0.4598027, 0.09773029, 0.08703377, 0.08668958...",IMG2744
4,IMG2744_5,-9.380678,3.58272,[[176.59672634 180.22641643 181.04637214]\n [ ...,"[0.5870208333333333, 0.24822083333333333, 0.16...","[cliff_dwelling, obelisk, beacon, bell_cote, t...","[0.41137695, 0.36667395, 0.047409043, 0.031869...",IMG2744
...,...,...,...,...,...,...,...,...
7495,IMG4243_1,-4.680678,35.18272,[[128.27104488 134.23647568 138.81402315]\n [1...,"[0.6565333333333333, 0.0879625, 0.255504166666...","[sliding_door, bannister, cinema, dining_table...","[0.14514625, 0.14219695, 0.05935542, 0.0578398...",IMG4243
7496,IMG4243_2,-4.680678,35.18272,[[118.48179231 110.87987609 77.6406008 ]\n [1...,"[0.7056708333333334, 0.11548333333333333, 0.17...","[altar, tray, shield, throne, chest]","[0.8379609, 0.045215227, 0.021772249, 0.013163...",IMG4243
7497,IMG4243_3,-4.680678,35.18272,[[226.50473934 221.93992891 217.16848341]\n [1...,"[0.5093708333333333, 0.4554625, 0.035166666666...","[church, cinema, altar, monastery, throne]","[0.22607179, 0.18696362, 0.12684114, 0.1066632...",IMG4243
7498,IMG4243_4,-4.680678,35.18272,[[133.48818234 141.77044155 145.40597314]\n [ ...,"[0.20084166666666667, 0.21566666666666667, 0.5...","[home_theater, shoji, wardrobe, bannister, sli...","[0.19004883, 0.16281106, 0.07418765, 0.0505939...",IMG4243


In [60]:
train_color = train_color.drop(columns=['Unnamed: 0'])

In [54]:
test_color

,Unnamed: 0,id,color,weight,object,object_weight
0,0,IMG4287_3,[[ 73.6183987 65.63426149 48.97455157]\n [1...,"[0.4972, 0.23238333333333333, 0.2704166666666667]","[sliding_door, bookcase, medicine_chest, china...","[0.22203007, 0.07611684, 0.07113481, 0.0683116..."
1,1,IMG4288_5,[[137.14717413 138.33202102 138.31306933]\n [2...,"[0.5061833333333333, 0.17682083333333334, 0.31...","[window_shade, bannister, binoculars, sliding_...","[0.2997648, 0.24434192, 0.10294387, 0.09043626..."
2,2,IMG4289_5,[[175.80050757 183.23411583 156.67026194]\n [1...,"[0.33195416666666666, 0.6221166666666667, 0.04...","[wardrobe, chiffonier, vestment, throne, organ]","[0.95141023, 0.016296614, 0.008123173, 0.00781..."
3,3,IMG4290_4,[[ 77.09971878 81.18058383 43.14020676]\n [2...,"[0.07971666666666667, 0.3940458333333333, 0.52...","[lakeside, breakwater, mobile_home, recreation...","[0.16709629, 0.08717257, 0.059366558, 0.043662..."
4,4,IMG4291_5,[[119.37971298 122.01727819 122.79116501]\n [1...,"[0.5002416666666667, 0.3787375, 0.121020833333...","[missile, bannister, projectile, patio, planet...","[0.5303798, 0.13834873, 0.109011896, 0.0902504..."
...,...,...,...,...,...,...
1195,1195,IMG5482_1,[[129.78609983 127.33521057 123.53794097]\n [1...,"[0.21656666666666666, 0.35668333333333335, 0.4...","[patio, washbasin, umbrella, monastery, vault]","[0.11151403, 0.10487429, 0.09955771, 0.0700328..."
1196,1196,IMG5483_2,[[114.87391959 107.25219356 101.11700607]\n [ ...,"[0.7636041666666666, 0.08890833333333334, 0.14...","[medicine_chest, switch, vending_machine, file...","[0.3165669, 0.25459656, 0.11216868, 0.06163487..."
1197,1197,IMG5484_4,[[162.45384074 169.47622777 181.3246514 ]\n [ ...,"[0.227375, 0.6331041666666667, 0.1395208333333...","[patio, greenhouse, solar_dish, bannister, pic...","[0.41546175, 0.20096128, 0.11161929, 0.0728891..."
1198,1198,IMG5485_3,[[ 88.79073834 53.05914677 19.60230792]\n [1...,"[0.8742583333333334, 0.08340416666666667, 0.04...","[restaurant, throne, altar, grand_piano, vault]","[0.14357524, 0.14155301, 0.08478197, 0.0836501..."


In [29]:
#for i in range(len(train)):
train_object = []
train_weight = []
for i in tqdm(range(len(train))):
    imgpath = train_path+"/"+train[i]
    img = image.load_img(imgpath, target_size=(224, 224))
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    k_img = preprocess_input(img)
    preds = model.predict(k_img)

    
    detect_object = [pred[1] for pred in decode_predictions(preds, top=5)[0]]
    detect_weight = [pred[2] for pred in decode_predictions(preds, top=5)[0]]

    train_object.append(detect_object)
    train_weight.append(detect_weight)




100%|██████████| 7500/7500 [06:16<00:00, 19.93it/s]


In [30]:
import pandas as pd

In [31]:
with_object_train = pd.DataFrame({"object":train_object,"object_weight":train_weight})

In [37]:
traindata = pd.read_csv("data/train.csv")
train_cluster = pd.read_csv("data/with_cluster_train.csv")
train_color = pd.read_csv("data/with_color_train.csv")

In [39]:
train_color = train_color.rename(columns={'weight': 'color_weight'})

In [41]:
train_color["object"] = with_object_train["object"]
train_color["object_weight"] = with_object_train["object_weight"]
train_color["cluster"] = train_cluster["cluster"]

In [44]:
train_color.columns

Index(['Unnamed: 0', 'id', 'x', 'y', 'color', 'color_weight', 'object',
       'object_weight', 'cluster'],
      dtype='object')

In [46]:
train_color.drop(columns=['Unnamed: 0'])

,id,x,y,color,color_weight,object,object_weight,cluster
0,IMG2744_1,-9.380678,3.58272,[[105.42161563 107.14448812 100.08701536]\n [ ...,"[0.6348333333333334, 0.28917916666666665, 0.07...","[patio, lakeside, bannister, mobile_home, brass]","[0.5601092, 0.14016706, 0.022413984, 0.0214102...",IMG2744
1,IMG2744_2,-9.380678,3.58272,[[ 70.89338335 69.35952808 46.43450226]\n [2...,"[0.24418333333333334, 0.383675, 0.372141666666...","[lakeside, fountain, snowplow, cab, patio]","[0.19650029, 0.09605682, 0.07403735, 0.0641981...",IMG2744
2,IMG2744_3,-9.380678,3.58272,[[102.4665223 104.16690078 106.75012681]\n [ ...,"[0.38762083333333336, 0.21436666666666668, 0.3...","[planetarium, recreational_vehicle, patio, dom...","[0.19081369, 0.18522698, 0.15903644, 0.1425865...",IMG2744
3,IMG2744_4,-9.380678,3.58272,[[172.34591855 173.62166457 174.12597505]\n [ ...,"[0.14006666666666667, 0.80195, 0.0579833333333...","[patio, prison, vault, sliding_door, library]","[0.4598027, 0.09773029, 0.08703377, 0.08668958...",IMG2744
4,IMG2744_5,-9.380678,3.58272,[[176.59672634 180.22641643 181.04637214]\n [ ...,"[0.5870208333333333, 0.24822083333333333, 0.16...","[cliff_dwelling, obelisk, beacon, bell_cote, t...","[0.41137695, 0.36667395, 0.047409043, 0.031869...",IMG2744
...,...,...,...,...,...,...,...,...
7495,IMG4243_1,-4.680678,35.18272,[[128.27104488 134.23647568 138.81402315]\n [1...,"[0.6565333333333333, 0.0879625, 0.255504166666...","[sliding_door, bannister, cinema, dining_table...","[0.14514625, 0.14219695, 0.05935542, 0.0578398...",IMG4243
7496,IMG4243_2,-4.680678,35.18272,[[118.48179231 110.87987609 77.6406008 ]\n [1...,"[0.7056708333333334, 0.11548333333333333, 0.17...","[altar, tray, shield, throne, chest]","[0.8379609, 0.045215227, 0.021772249, 0.013163...",IMG4243
7497,IMG4243_3,-4.680678,35.18272,[[226.50473934 221.93992891 217.16848341]\n [1...,"[0.5093708333333333, 0.4554625, 0.035166666666...","[church, cinema, altar, monastery, throne]","[0.22607179, 0.18696362, 0.12684114, 0.1066632...",IMG4243
7498,IMG4243_4,-4.680678,35.18272,[[133.48818234 141.77044155 145.40597314]\n [ ...,"[0.20084166666666667, 0.21566666666666667, 0.5...","[home_theater, shoji, wardrobe, bannister, sli...","[0.19004883, 0.16281106, 0.07418765, 0.0505939...",IMG4243


In [49]:
train_color.to_csv("data/train_features.csv",index = False)

In [22]:
imgpath = train_path+"/"+train[2]
img = image.load_img(imgpath, target_size=(224, 224))
# convert the image pixels to a numpy array
img = image.img_to_array(img)
# reshape data for the model
img = np.expand_dims(img, axis=0)
# convert the input to 4D input makes it the same as the training process,
# with a placeholder, rows, cols, the number of colour channels as training process
# axis: Position in the expanded axes where the new axis (or axes) is placed.
# axis = 0: insert axis at position 0 

# prepare the image for the model
k_img = preprocess_input(img)
# predict the probability across all output classes
# model.predict(input) return the same data structure as your input
preds = model.predict(k_img)

#print('wallaby prediction:', decode_predictions(preds, top=5)[0])

print('wallaby prediction:')
# convert the probabilities to class labels and retrieve the top 5 highest probability result
for pred in decode_predictions(preds, top=5)[0]: 
    # print as labels: probability
    print('%s: %.3f%%' % (pred[1], pred[2]*100))

# display the image
# im = plt.imread(imgpath)
# plt.imshow(im)
# plt.axis('off')
# plt.show()

wallaby prediction:
planetarium: 19.081%
recreational_vehicle: 18.523%
patio: 15.904%
dome: 14.259%
mobile_home: 6.679%


In [19]:
decode_predictions(preds, top=5)

[[('n03899768', 'patio', 0.5601092),
  ('n09332890', 'lakeside', 0.14016706),
  ('n02788148', 'bannister', 0.022413984),
  ('n03776460', 'mobile_home', 0.021410258),
  ('n02892201', 'brass', 0.01758092)]]